<a href="https://colab.research.google.com/github/goerlitz/nlp-classification/blob/main/notebooks/10kGNAD/colab/21c2_10kGNAD_huggingface_optuna_config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Optimization with HuggingFace Transformers

Adapted from https://huggingface.co/docs/transformers/custom_datasets#sequence-classification-with-imdb-reviews

Things we need
* a tokenizer
* tokenized input data
* a pretrained model
* evaluation metrics
* training parameters
* a Trainer instance

Notes
* [class labels can be included in the model config](https://github.com/huggingface/transformers/pull/2945#issuecomment-781986506) (a bit hacky)
* [fp16 is disabled on tesla P100 GPU in pytorch](https://discuss.pytorch.org/t/cnn-fp16-slower-than-fp32-on-tesla-p100/12146)
* [comparison of GPUS (K80, T4, P100, V100)](https://www.kaggle.com/general/198232)
* [GPU benchmark, mixed precision](https://medium.com/the-artificial-impostor/mixed-precision-training-on-tesla-t4-and-p100-d82e5d3b987d)

## Prerequisites

In [ ]:
# checkpoint = "distilbert-base-german-cased"
checkpoint = "deepset/gbert-base"
# checkpoint = "deepset/gelectra-base"
# checkpoint = "deepset/gelectra-large"

project_name = f'10kgnad_hf__{checkpoint.replace("/", "_")}'

### Connect Google Drive

Will be used to save results

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
from pathlib import Path

# define model path
root_path = Path('/content/gdrive/My Drive/')
base_path = root_path / 'Colab Notebooks/nlp-classification/'
model_path = base_path / 'models'

## Check GPU

In [15]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jan 30 22:50:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    35W / 250W |   3899MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install APEX

https://stackoverflow.com/questions/57284345/how-to-install-nvidia-apex-on-google-colab

In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -q --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex >/dev/null

Overwriting setup.sh


In [ ]:
%%time
# !sh setup.sh

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


### Install Packages

In [ ]:
%%time
!pip install -q -U transformers datasets >/dev/null
!pip install -q -U optuna >/dev/null

# check installed version
!pip freeze | grep optuna        # optuna==2.10.0
!pip freeze | grep transformers  # transformers==4.15.0
!pip freeze | grep "torch "      # torch==1.10.0+cu111

optuna==2.10.0
transformers==4.16.1
torch @ https://download.pytorch.org/whl/cu111/torch-1.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
CPU times: user 134 ms, sys: 448 ms, total: 582 ms
Wall time: 16.4 s


In [ ]:
from transformers import logging

# hide progress bar when downloading tokenizer and model (a workaround!)
logging.get_verbosity = lambda : logging.NOTSET

## Load Dataset

In [ ]:
from datasets import load_dataset

base_url = "https://raw.githubusercontent.com/tblock/10kGNAD/master/{}.csv"
data_files = {x: base_url.format(x) for x in ["train", "test"]}
dataset = (load_dataset('csv',
                        data_files=data_files,
                        sep=";",
                        quotechar="'",
                        names=["label", "text"]).
           class_encode_column("label"))

label_names = dataset["train"].features["label"].names

Using custom data configuration default-0e1a53e9f937c1cf
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-7e2cd654f77312b3.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-8a8200b7f43f1260.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-575eed89dcd8fbbd.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-0e1a53e9f937c1cf/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-944af939de8444f9.arrow


In [ ]:
print(dataset)
print("labels:", label_names)
dataset["train"][0]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 9245
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1028
    })
})
labels: ['Etat', 'Inland', 'International', 'Kultur', 'Panorama', 'Sport', 'Web', 'Wirtschaft', 'Wissenschaft']


{'label': 5,
 'text': '21-Jähriger fällt wohl bis Saisonende aus. Wien – Rapid muss wohl bis Saisonende auf Offensivspieler Thomas Murg verzichten. Der im Winter aus Ried gekommene 21-Jährige erlitt beim 0:4-Heimdebakel gegen Admira Wacker Mödling am Samstag einen Teilriss des Innenbandes im linken Knie, wie eine Magnetresonanz-Untersuchung am Donnerstag ergab. Murg erhielt eine Schiene, muss aber nicht operiert werden. Dennoch steht ihm eine mehrwöchige Pause bevor.'}

### Use Dynamic Padding

Apply panding only on longest text in batch - this is more efficient than applying padding on the whole dataset.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Define Evaluation Metrics

The funtion that computes the metrics needs to be passed to the Trainer.

## Hyperparameter Tuning

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import shutil

def hp_space(trial: Trial):
    """A function that defines the hyperparameter search space.
    To be used in :obj:`Trainer.hyperparameter_search`."""
    return {
        # "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True),  # distilbert/bert
        "learning_rate": trial.suggest_float("learning_rate", 5e-6, 5e-4, log=True),  # distilbert 1 epoch
        # "learning_rate": trial.suggest_float("learning_rate", 6e-5, 2e-4, log=True),  # electra
        # "num_train_epochs": trial.suggest_categorical("num_train_epochs", [1]),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [3]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32, 64, 128, 256]),
        "weight_decay": trial.suggest_float("weight_decay", 1e-3, 1e-2, log=True),
        # "weight_decay": trial.suggest_categorical("weight_decay", [1e-3, 0.0]),
    }

best_model_dir = "best_model_trainer"

def best_model_callback(study, trial):
    """Save the model from a best trial"""
    for t in study.best_trials:
        if t.number == trial.number:
            print("This is a new besttrial", trial.number)
        
            out_filename = model_path / f"{project_name}_t{trial.number}"
            shutil.make_archive(out_filename, 'zip', f"{project_name}/{best_model_dir}")

def model_init(trial: Trial):
    """A function that instantiates the model to be used."""

    # We want to include the label names and save them together with the model.
    # The only way to do this is to create a Config and put them in. 
    config = AutoConfig.from_pretrained(
            checkpoint,
            num_labels=len(label_names),
            id2label={i: label for i, label in enumerate(label_names)},
            label2id={label: i for i, label in enumerate(label_names)},
            )

    return AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

def objective(trial: Trial):

    # get hyperparameters choice
    hp = hp_space(trial)
    lr = hp["learning_rate"]
    bs = hp["per_device_train_batch_size"]
    epochs = hp["num_train_epochs"]
    weight_decay = hp["weight_decay"]
    # label_smoothing_factor = hp["label_smoothing_factor"]

    # calculate gradient_accumulation_steps
    train_batch_size = 8
    gradient_accumulation_steps = bs // train_batch_size

    eval_rounds_per_epoch = 5
    eval_steps = dataset["train"].num_rows / bs // eval_rounds_per_epoch

    training_args = TrainingArguments(
        output_dir=str(project_name),
        report_to=[],
        log_level="error",
        disable_tqdm=False,

        evaluation_strategy="steps",
        eval_steps=eval_steps,
        logging_steps=eval_steps,
        save_strategy="steps",
        save_steps=eval_steps,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,

        # hyperparameters
        num_train_epochs=epochs,
        learning_rate=lr,
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        weight_decay=weight_decay,
        # label_smoothing_factor=label_smoothing_factor,

        # fp16=True,  # fp16 needs apex. but disabled on Tesla P100 by pytorch
    )

    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        # data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[TrialLogAndPruningCallback(trial, objectives=["eval_loss", "eval_f1"], min_trials=700, warmup_steps=eval_steps*3)]
        # callbacks=[TrialPruningCallback(trial)]
    )

    # train model and save best model from evaluations
    # needs 'load_best_model_at_end=True'
    trainer.train()
    trainer.save_model(f"{project_name}/{best_model_dir}")

    result = trainer.evaluate(eval_dataset=tokenized_dataset["test"])

    # store eval metrics in trial
    trial.set_user_attr("eval_result", result)
    
    # return result["eval_loss"]
    return result["eval_loss"], result["eval_f1"]

## Hyperparameter Tuning

In [ ]:
import optuna
from optuna.storages import RDBStorage
import random
import numpy as np

db_path = "/content/gdrive/My Drive/Colab Notebooks/nlp-classification/"
db_name = "10kgnad_optuna"
# study_name = checkpoint + "_multi_epoch234"
# study_name = checkpoint + "_loss-f1_bs32_epoch23"
# study_name = checkpoint + "_loss-f1_bs32_ep3_pad"
study_name = checkpoint + "_bs8-256_ep3_len128"

# automatically change the state of a stale trial to TrialState.FAIL from TrialState.RUNNING
storage = RDBStorage(url=f"sqlite:///{db_path}{db_name}.db", heartbeat_interval=60, grace_period=120)

# https://stackoverflow.com/questions/59129812/how-to-avoid-cuda-out-of-memory-in-pytorch
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

# multi objective study
# https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/002_multi_objective.html#sphx-glr-tutorial-20-recipes-002-multi-objective-py
study = optuna.create_study(study_name=study_name,
                            directions=["minimize", "maximize"],
                            # storage=f"sqlite:///{db_path}{db_name}.db",
                            storage=storage,
                            load_if_exists=True,)

# ------ prime with parameters
def lr_sample(min, max, dist=0.1, jitter=0.1):
    min_log = np.log10(min)
    max_log = np.log10(max)
    n = int((max_log - min_log) / dist)
    return np.logspace(min_log, max_log, n) * np.random.uniform(1-jitter, 1+jitter, size=n)

def lr_pairs():
    lrs = []
    min_lr = 5e-6
    max_lr = 5e-4
    for bs in [8, 16, 32, 64, 128, 256]:
        lrs.extend((bs, lr) for lr in lr_sample(min_lr, max_lr, dist=0.05, jitter=0.05))
    random.shuffle(lrs)
    return lrs

for bs, lr in lr_pairs():
    study.enqueue_trial(
        {
                "learning_rate": lr,
                "per_device_train_batch_size": bs,
            }
        )

# give some hyperparameters that are presumably good
# for bs in [8,16,32,64,128]:
#     for lr in np.exp(np.linspace(np.log(7e-6), np.log(2e-4), 15)):
#         study.enqueue_trial(
#             {
#                 "learning_rate": lr,
#                 "per_device_train_batch_size": bs,
#             }
#         )
# study.enqueue_trial(
#     {
#         "learning_rate": 5.1e-5,
#         "per_device_train_batch_size": 32,
#     }
# )
# study.enqueue_trial(
#     {
#         "learning_rate": 5.8e-5,
#         "per_device_train_batch_size": 32,
#     }
# )


study.optimize(objective, n_trials=200, callbacks=[best_model_callback])

# study.best_params

[I 2022-01-29 07:46:30,994] A new study created in RDB with name: deepset/gbert-base_bs8-256_ep3_len128
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: ExperimentalWarning: enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.
/usr/local/lib/python3.7/dist-packages/optuna/study/study.py:857: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})
/usr/local/lib/python3.7/dist-packages/optuna/study/study.py:857: ExperimentalWarning: add_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})
fixed params: [('num_train_epochs', 3), ('per_device_train_batch_size', 16)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=

Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall,Mcc
115,1.537100,0.897237,0.756809,0.716140,0.813256,0.700500,0.721879
230,0.738400,0.534543,0.857004,0.849835,0.853672,0.848348,0.836384
345,0.544000,0.459638,0.860895,0.854224,0.852459,0.859615,0.841169
460,0.473800,0.437892,0.870623,0.864380,0.862016,0.872975,0.852892
575,0.455700,0.399327,0.887160,0.883708,0.889948,0.879312,0.870744
690,0.371600,0.380467,0.877432,0.873802,0.877573,0.871174,0.859624
805,0.303700,0.365358,0.892996,0.886700,0.887809,0.886420,0.877468
920,0.311700,0.386337,0.875486,0.869010,0.875974,0.866185,0.857515
1035,0.319400,0.374001,0.892023,0.884317,0.883084,0.888582,0.876567
1150,0.344200,0.368000,0.893969,0.886957,0.890175,0.886085,0.878631


[I 2022-01-29 07:56:30,721] Trial 0 finished with values: [0.3627486228942871, 0.8901865223926281] and parameters: {'learning_rate': 9.170262586219904e-06, 'num_train_epochs': 3, 'per_device_train_batch_size': 16, 'weight_decay': 0.0027599208115882986}. 


This is a new besttrial 0


fixed params: [('num_train_epochs', 3), ('per_device_train_batch_size', 32)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=171.0, min_trials=700
params: {'learning_rate': 0.00025377987451239765, 'num_train_epochs': 3, 'per_device_train_batch_size': 32, 'weight_decay': 0.0027855104091463493}
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2022-01-29 13:50:38,467] Trial 38 finished with values: [1.2240041494369507, 0.54046694494783] and parameters: {'learning_rate': 5.392686865617486e-06, 'num_train_epochs': 3, 'per_device_train_batch_size': 256, 'weight_decay': 0.004339891033507396}. 
fixed params: [('num_train_epochs', 3), ('per_device_train_batch_size', 32)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=171.0, min_trials=700
params: {'learning_rate': 0.00014702055839263257, 'num_train_epochs': 3, 'per_device_train_batch_size': 32, 'weight_decay': 0.001630971369462351}
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarni

Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall,Mcc
57,0.898000,0.660651,0.798638,0.785095,0.793563,0.806144,0.773463
114,0.570400,0.525529,0.828794,0.822005,0.837418,0.822530,0.804911
171,0.565000,0.471834,0.856031,0.846651,0.847570,0.857199,0.836550
228,0.476600,0.456308,0.859922,0.854037,0.856160,0.863981,0.840907
285,0.548900,0.414406,0.879377,0.875806,0.881667,0.871774,0.861747
342,0.324600,0.464491,0.866732,0.853801,0.859490,0.856289,0.847959
399,0.254000,0.446613,0.872568,0.866990,0.869521,0.870346,0.854727
456,0.297000,0.423336,0.880350,0.872010,0.876185,0.872445,0.863396
513,0.249200,0.409162,0.890078,0.883529,0.882581,0.890833,0.875127
570,0.301200,0.376262,0.891051,0.887757,0.892161,0.884052,0.875117


[I 2022-01-29 13:59:48,269] Trial 39 finished with values: [0.37626218795776367, 0.8877572152080605] and parameters: {'learning_rate': 0.00014702055839263257, 'num_train_epochs': 3, 'per_device_train_batch_size': 32, 'weight_decay': 0.001630971369462351}. 
fixed params: [('num_train_epochs', 3), ('per_device_train_batch_size', 64)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=84.0, min_trials=700
params: {'learning_rate': 4.131270983914418e-05, 'num_train_epochs': 3, 'per_device_train_batch_size': 64, 'weight_decay': 0.0012996858167884208}
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall,Mcc
28,1.362900,0.675870,0.818093,0.799567,0.836014,0.785481,0.792426
56,0.593600,0.461886,0.860895,0.855798,0.859355,0.860816,0.841852
84,0.477500,0.419125,0.866732,0.859106,0.871995,0.856416,0.848141
112,0.431200,0.379825,0.885214,0.877976,0.878065,0.882681,0.869133
140,0.447100,0.372712,0.884241,0.878633,0.875935,0.883313,0.867716
168,0.297800,0.352463,0.882296,0.878974,0.882626,0.876395,0.865196
196,0.234600,0.347910,0.886187,0.879943,0.875481,0.885816,0.869860
224,0.256600,0.341687,0.890078,0.884916,0.882210,0.889309,0.874318
252,0.250600,0.333345,0.893969,0.889003,0.892741,0.886495,0.878534
280,0.266300,0.329047,0.898833,0.893913,0.896075,0.892224,0.884057


In [ ]:
!ls -lahtr $project_name

---
## New Optimization Code

In [1]:
!pip install -q optuna transformers datasets >/dev/null

In [9]:
from transformers import TrainerCallback
from optuna.trial import Trial, TrialState
from optuna.study._study_direction import StudyDirection
import pandas as pd

# https://github.com/huggingface/transformers/blob/v4.14.1/src/transformers/trainer_callback.py#L505
# https://huggingface.co/docs/transformers/main_classes/callback#transformers.TrainerCallback

import logging
logging.getLogger(__name__).setLevel(logging.INFO)
log = logging.getLogger(__name__)

class TrialLogAndPruningCallback(TrainerCallback):
    """Stores eval metrics at each evaluation step in the trial user attrs."""
    def __init__(self, trial: Trial, objectives=None, warmup_steps=0, min_trials=7):
        self.study = trial.study
        self.trial = trial
        self.param_keys = ["num_train_epochs", "batch_size"]
        self.param_vals = [trial.params[k] for k in self.param_keys]

        log.warning(f"fixed params: {list(zip(self.param_keys, self.param_vals))}")

        if objectives == None:
            self.objectives = ["eval_loss"]
        else:
            self.objectives = objectives
        self._warmup_steps = warmup_steps
        self._min_trials = max(1, int(min_trials))

        log.warning(f"objectives: {self.objectives}, directions: {self.study.directions}, warmup={self._warmup_steps}, min_trials={self._min_trials}")
        log.warning(f"params: {trial.params}")
        

    def _filter_trials(self, complete_trials):
        """Select only trials with same parameter values"""
        # values = [self.trial.params[k] for k in keys]
        return [t for t in complete_trials if self.param_vals == [t.params[k] for k in self.param_keys]]

    def _prune(self, step: int, metrics) -> bool:
        """Median Pruning on multiple objectives."""
        if step < self._warmup_steps:
            # log.warning(f"less than warmup steps {step}<{self._warmup_steps}")
            return False

        # get all completed trials
        complete_trials = self.study.get_trials(deepcopy=False,
                                                states=[TrialState.COMPLETE])
        # only compare trials with same batch size and epochs
        complete_trials = self._filter_trials(complete_trials)
        n_trials = len(complete_trials)

        # check minimal number of trial required
        if n_trials < self._min_trials:
            # log.warning(f"less than min trials {n_trials}<{self._min_trials}")
            return False

        # log.warning(f"checking {step}: {metrics}")

        # sanity check
        has_metrics = [o in metrics.keys() for o in self.objectives]
        if not all(has_metrics):
            log.warning(f"missing objective metrics {list(zip(self.objectives, has_metrics))}")

        # extract metrics from trials
        # print(f"fetching metrics of {n_trials} complete trials")
        trial_metrics = []
        for t in complete_trials:
            # print(str(step), "in keys?", str(step) in t.user_attrs.keys(), t.user_attrs.keys())
            if str(step) in t.user_attrs.keys():
                trial_metrics.append(t.user_attrs[str(step)])
        n_metrics = len(trial_metrics)

        # compute median for each metric over all trials
        median = pd.DataFrame(trial_metrics).median()

        # log.warning(f"median of {n_metrics}/{n_trials}: {median.to_dict()}")

        # compare current metric value with median
        prune_state = []
        for i, o in enumerate(self.objectives):
            if self.study.directions[i] == StudyDirection.MAXIMIZE:
                prune_state.append(metrics[o] <= median[o])
            else:
                prune_state.append(metrics[o] > median[o])
        
        met = ",".join([f"{m}={metrics[m]:.4}/{median[m]:.4}" for m in self.objectives])
        print(f"prune? step={step}, warmup={self._warmup_steps}, complete_trials={n_trials}, metrics={n_metrics} -> {met}; {prune_state}")

        # all metrics must be marked for pruning
        return all(prune_state)
    
    def on_evaluate(self, args, state, control, lr_scheduler, metrics, **kwargs):
        step = state.global_step
        values = {**metrics, "lr": lr_scheduler.get_last_lr()[-1]}
        self.trial.set_user_attr(str(step), values)

        # pruning
        if self._prune(step, metrics):
            print(f"pruning trial at step {step}")
            # control.should_training_stop = True  # not needed
            raise optuna.TrialPruned()

In [10]:
import optuna

class Interval:
    def __init__(self, min, max, log=False):
        self.min = min
        self.max = max
        self.log = log

def uniform(min, max):
    return Interval(min, max, log=False)

def log_uniform(min, max):
    return Interval(min, max, log=True)

class Space:

    def __init__(self, **kwargs):
        self.params = kwargs
    
    def _suggest(self, trial, key, val):
        if val is None or isinstance(val, (bool, float, int, str)):
            return trial.suggest_categorical(key, [val])
        if isinstance(val, list):
            return trial.suggest_categorical(key, val)
        if isinstance(val, Interval):
            return trial.suggest_float(key, val.min, val.max, log=val.log)

    def suggest(self, trial: optuna.trial.Trial):
        return {k:self._suggest(trial, k, v) for k, v in self.params.items()}


from transformers import AutoTokenizer
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import logging as trlog
from datasets import DatasetDict, Dataset, load_dataset
from datasets import logging as dslog
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, matthews_corrcoef, precision_recall_fscore_support
import numpy as np

# hide progress bar when downloading dataset - needs workaround!
dslog.get_verbosity = lambda : logging.NOTSET
trlog.get_verbosity = lambda : logging.NOTSET

def load_data():
    base_url = "https://raw.githubusercontent.com/tblock/10kGNAD/master/{}.csv"
    data_files = {x: base_url.format(x) for x in ["train", "test"]}
    dataset = (load_dataset('csv',
                            data_files=data_files,
                            sep=";",
                            quotechar="'",
                            names=["label", "text"]).
            class_encode_column("label"))
    label_names = dataset["train"].features["label"].names
    return dataset, label_names


def sample_data(ds: DatasetDict, train_size: float, columns=["text", "label"]):
    """Create a stratified sample of the train dataset."""
    X = ds["train"][columns[0]]
    y = ds["train"][columns[1]]
    X_train, _, y_train, _ = train_test_split(X, y, train_size=train_size, random_state=42, stratify=y)
    return DatasetDict({
        "train": Dataset.from_dict({"label": y_train, "text": X_train}),
        "test": ds["test"]
    })

data_cache = {}

def prepare_data(tokenizer, params) -> DatasetDict:

    train_size = params["train_size"]
    max_seq_length = params["max_seq_length"]
    identifier = f"{train_size}/{max_seq_length}"

    if identifier in data_cache:
        print(f"USING CACHED DATASET for {identifier}")
        return data_cache[identifier]
    else:
        print("GENERATING DATASET")

    dataset, label_names = load_data()

    if train_size is not None and train_size < 1.0:
        dataset = sample_data(dataset, train_size)

    if max_seq_length is None:
        max_seq_length = getattr(tokenizer, "model_max_length")
    
    # TODO better use a partial function
    def preprocess_function(examples):
        return tokenizer(examples["text"], truncation=True, max_length=max_seq_length)
    
    mapped_data = dataset.map(preprocess_function, batched=True).remove_columns("text")

    print("STORING DATASET")
    data_cache[identifier] = (mapped_data, label_names)
    return mapped_data, label_names


def create_training_args(params: dict, config: dict) -> TrainingArguments:
    
    # output_dir = config["output_dir"]
    train_rows = config["train_rows"]
    base_batch_size = config["base_batch_size"]
    eval_rounds_per_epoch = config["eval_rounds_per_epoch"]

    # calculate gradient_accumulation and evaluation steps
    bs = params["batch_size"]
    gradient_accumulation_steps = bs // base_batch_size
    eval_steps = train_rows / bs // eval_rounds_per_epoch
    if (train_rows / bs < eval_rounds_per_epoch):
        raise ValueError(f"batch size {bs} is too big for {train_rows} examples and {eval_rounds_per_epoch} eval rounds!")

    return TrainingArguments(
        output_dir=config["output_dir"],
        report_to=[],
        log_level="error",
        disable_tqdm=False,

        evaluation_strategy="steps",
        eval_steps=eval_steps,
        logging_steps=eval_steps,
        save_strategy="steps",
        save_steps=eval_steps,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,

        # hyperparameters
        num_train_epochs=params["num_train_epochs"],
        learning_rate=params["learning_rate"],
        per_device_train_batch_size=base_batch_size,
        per_device_eval_batch_size=base_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        weight_decay=params["weight_decay"],

        # fp16=True,  # fp16 needs apex. but disabled on Tesla P100 by pytorch
    )


def init_model(checkpoint, label_names):
    """A function that instantiates the model to be used."""

    # We want to include the label names and save them together with the model.
    # The only way to do this is to create a Config and put them in. 
    config = AutoConfig.from_pretrained(
            checkpoint,
            num_labels=len(label_names),
            id2label={i: label for i, label in enumerate(label_names)},
            label2id={label: i for i, label in enumerate(label_names)},
            )

    return AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

def compute_metrics(eval_preds):
    """The function that will be used to compute metrics at evaluation.
    Must take a :class:`~transformers.EvalPrediction` and return a dictionary
    string to metric values."""
    logits, labels = eval_preds
    preds = np.argmax(logits, axis=-1)
    # precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average='macro'),
        "precision": precision_score(labels, preds, average='macro'),
        "recall": recall_score(labels, preds, average='macro'),
        "mcc": matthews_corrcoef(labels, preds),
        }

def objective(trial):

    # suggest hyperparameters
    hp = space.suggest(trial)
    print(hp)

    checkpoint = hp["model"]
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    # prepare dataset
    tokenized_dataset, label_names = prepare_data(tokenizer, hp)
    # print(f"LABEL NAMES {label_names}")


    project_name = "test"
    best_model_dir = "best_model"

    config = {
        "output_dir": project_name,
        "base_batch_size": 8,
        "eval_rounds_per_epoch": 5,
        "train_rows": tokenized_dataset["train"].num_rows
    }

    ## TODO: calculate batch size and aggregations steps separately

    # create training args
    training_args = create_training_args(hp, config)
    # print(args)

    # https://stackoverflow.com/questions/59129812/how-to-avoid-cuda-out-of-memory-in-pytorch
    import torch
    torch.cuda.empty_cache()
    import gc
    gc.collect()


    # prepare Trainer
    trainer = Trainer(
        model_init=lambda x: init_model(checkpoint, label_names),
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        # data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[TrialLogAndPruningCallback(trial, objectives=["eval_loss", "eval_f1"], min_trials=700, warmup_steps=training_args.eval_steps*3)]
    )

    # train model and save best model from evaluations
    # needs 'load_best_model_at_end=True'
    trainer.train()
    trainer.save_model(f"{project_name}/{best_model_dir}")

    result = trainer.evaluate(eval_dataset=tokenized_dataset["test"])

    # store eval metrics in trial
    trial.set_user_attr("eval_result", result)
    
    # return result["eval_loss"]
    return result["eval_loss"], result["eval_f1"]

import random
import numpy as np

def lr_sample(min, max, dist=0.1, jitter=0.1):
    min_log = np.log10(min)
    max_log = np.log10(max)
    n = int((max_log - min_log) / dist)
    return np.logspace(min_log, max_log, n) * np.random.uniform(1-jitter, 1+jitter, size=n)

def lr_pairs(min_lr, max_lr, hp_values, dist=0.1, jitter=0.1, shuffle=True):
    lrs = []
    for val in hp_values:
        lrs.extend((val, lr) for lr in lr_sample(min_lr, max_lr, dist=dist, jitter=jitter))
    if shuffle:
        random.shuffle(lrs)
    return lrs

def prepare_trials(study, min_lr, max_lr, hp_name, hp_values, dist=0.1, jitter=0.1, shuffle=True):
    # ------ prime with parameters

    for val, lr in lr_pairs(min_lr, max_lr, hp_values, dist=0.1, jitter=0.1, shuffle=shuffle):
        study.enqueue_trial(
            {
                    "learning_rate": lr,
                    hp_name: val,
                }
            )

In [ ]:
from optuna.storages import RDBStorage

db_path = "/content/gdrive/My Drive/Colab Notebooks/nlp-classification/"
db_name = "10kgnad_optuna"
# automatically change the state of a stale trial to TrialState.FAIL from TrialState.RUNNING
storage = RDBStorage(url=f"sqlite:///{db_path}{db_name}.db", heartbeat_interval=60, grace_period=120)

space = Space(
    model = "deepset/gbert-base",
    train_size = [0.25, 0.5, 1.0],
    data_collator = False,
    max_seq_length = 128,
    batch_size = [64],
    num_train_epochs = 2,
    learning_rate = log_uniform(5e-6, 5e-4),
    weight_decay = log_uniform(1e-3, 1e-2),
)

# TODO define optimization metrics

study_name = space.params["model"] + "_ds25-100_bs64_ep2_len128"
print("STUDY", study_name)

# multi objective study
# https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/002_multi_objective.html#sphx-glr-tutorial-20-recipes-002-multi-objective-py
study = optuna.create_study(study_name=study_name,
                            directions=["minimize", "maximize"],
                            storage=storage,
                            load_if_exists=True,)

# prime trials
lr = space.params["learning_rate"]
key = "train_size"
# prepare_trials(study, lr.min, lr.max, key, space.params[key], dist=0.05, jitter=0.05)

study.optimize(objective, n_trials=3)

[I 2022-01-30 23:00:06,780] Using an existing study with name 'deepset/gbert-base_ds25-100_bs64_ep2_len128' instead of creating a new one.


STUDY deepset/gbert-base_ds25-100_bs64_ep2_len128


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:258: ExperimentalWarning: enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.
/usr/local/lib/python3.7/dist-packages/optuna/study/study.py:857: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})
/usr/local/lib/python3.7/dist-packages/optuna/study/study.py:857: ExperimentalWarning: add_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})


{'model': 'deepset/gbert-base', 'train_size': 0.25, 'data_collator': False, 'max_seq_length': 128, 'batch_size': 64, 'num_train_epochs': 2, 'learning_rate': 1.2276295732827837e-05, 'weight_decay': 0.0027240014343563727}
USING CACHED DATASET for 0.25/128


fixed params: [('num_train_epochs', 2), ('batch_size', 64)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=21.0, min_trials=700
params: {'model': 'deepset/gbert-base', 'train_size': 0.25, 'data_collator': False, 'max_seq_length': 128, 'batch_size': 64, 'num_train_epochs': 2, 'learning_rate': 1.2276295732827837e-05, 'weight_decay': 0.0027240014343563727}
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[I 2022-01-30 23:06:00,741] Trial 4 finished with values: [0.4870188534259796, 0.8495850771306106] and parameters: {'model': 'deepset/gbert-base', 'train_size': 0.25, 'data_collator': False, 'max_seq_length': 128, 'batch_size': 64, 'num_train_epochs': 2, 'learning_rate': 4.613658538714951e-05, 'weight_decay': 0.008837330426901786}. 


{'model': 'deepset/gbert-base', 'train_size': 1.0, 'data_collator': False, 'max_seq_length': 128, 'batch_size': 64, 'num_train_epochs': 2, 'learning_rate': 0.00011006711630444298, 'weight_decay': 0.0033087965136340096}
USING CACHED DATASET for 1.0/128


fixed params: [('num_train_epochs', 2), ('batch_size', 64)]
objectives: ['eval_loss', 'eval_f1'], directions: [<StudyDirection.MINIMIZE: 1>, <StudyDirection.MAXIMIZE: 2>], warmup=84.0, min_trials=700
params: {'model': 'deepset/gbert-base', 'train_size': 1.0, 'data_collator': False, 'max_seq_length': 128, 'batch_size': 64, 'num_train_epochs': 2, 'learning_rate': 0.00011006711630444298, 'weight_decay': 0.0033087965136340096}
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall,Mcc
28,1.068000,0.552434,0.832685,0.822455,0.844592,0.815563,0.810710
56,0.496300,0.446340,0.860895,0.857209,0.862571,0.859055,0.842293
84,0.453000,0.393846,0.873541,0.869858,0.869688,0.871670,0.855276
112,0.416800,0.413256,0.869650,0.865183,0.866581,0.869679,0.851737
140,0.447000,0.396629,0.880350,0.875680,0.876563,0.876831,0.863092
168,0.268500,0.366865,0.883268,0.879788,0.878630,0.883650,0.866637
